In [1]:
from arquivos import central
import matplotlib.pyplot as plt
from numba import njit
import numpy as np
import pandas as pd

In [2]:
robot = central.Central()

In [3]:
robot.get_normal_data_csv('C:\\Users\\johnk\\Google Drive\\My_Project\\Dados\\Data_D1_3k_icmarkets.csv', drop=True, drop_list=['Unnamed: 0'])

Dados Carregados com Sucesso.


In [4]:
%%time
robot.pct_data()
robot.balance_signal4()
df = robot.back_tpsl(10, 2, analyse=True)

Period data set D1
Wall time: 3.95 s


In [5]:
df

Analises,Largest_Win_€,Smallest_Win_€,Largest_Loss_€,Smallest_Loss_€,Annualized_Return_%,Average_Trades_Per_Year,Average_Win_Rate_%
Total_Trades,2.92,0.02,-0.07,-1.61,19.970773,161.841096,31.639355
Short_Trades,2.85,0.02,-0.07,-1.61,103.217770,78.846575,30.796762
Long_Trades,2.92,0.02,-0.07,-1.60,109.877225,82.991781,32.440908
EURCHF,0.95,0.73,-0.47,-0.58,81.234341,4.476712,47.001224
EURGBP,1.37,1.00,-0.60,-0.79,85.959509,4.824658,46.223737
EURJPY,NaN,NaN,-0.07,-0.07,-82.702581,8.216438,0.000000
EURNZD,0.64,0.46,-0.33,-0.43,79.145023,5.087671,49.596123
EURUSD,0.89,0.62,-0.41,-0.55,82.647809,4.983562,47.883452
EURAUD,0.79,0.46,-0.34,-0.50,89.056547,4.972603,60.330579
EURCAD,0.75,0.55,-0.38,-0.48,78.078727,5.021918,46.917621


In [18]:
df['Average_Trades_Per_Year'][3:].sum()

161.83835616438353

In [36]:
a

0.5

In [ ]:
for i in df:
    test['Max_Profit'] = df[i].max()
    test['Min_Profit'] = df[i].min()

In [ ]:
test = pd.DataFrame()
#test['Analises'] = ['Max_Profit', 'Min_Profit']
#test = test.set_index('Analises')
test['Analises'] = np.nan
test[df.columns] = np.nan
test.set_index('Analises', inplace=True)
test = test.T
test['Max_Profit'] = 10
test

In [37]:
1 / (len(robot.get_normal_data()) / 365)

0.12166666666666666

In [23]:
df['Total_Trades'].iloc[-1] / df['Total_Trades'].iloc[0] * 100

446.61699999988673

In [29]:
pow_ = 1 / (len(robot.get_normal_data()) / 365)

In [30]:
(((df['Total_Trades'].iloc[-1] / df['Total_Trades'].iloc[0]) ** pow_) - 1) * 100

19.970772528569626

In [32]:
(1 + 0.1997) ** 8.21

4.458476985640426